In [ ]:
# LOAD AND CLEAN PROVIDER INFO

In [198]:
import pandas as pd
import glob
import os
import copy

# Read files with different encodings
def read_csv_with_encodings(file_path):
    # List of common encodings to try
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'utf-16', 'utf-32']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding, low_memory = False)
            print(f"Successfully read the file with {encoding} encoding")
            return df
        except UnicodeDecodeError:
            print(f"Failed to read with {encoding} encoding")
    
    print("Could not read the file with any of the common encodings")
    return None

# Define the directory path
directory = '/Users/apple/Documents/APD/provider_info/original/'

# Get all CSV files matching the pattern
files = glob.glob(os.path.join(directory, 'ProviderInfo_*.csv'))

# Dictionaries to store original DataFrames
raw_provider_info = {}

# Process each file
for file_path in files:
    # Extract the year from the filename
    filename = os.path.basename(file_path)
    year = filename.replace('ProviderInfo_', '').replace('.csv', '')
    
    # Read the CSV file
    df = read_csv_with_encodings(file_path)
    
    if df is not None:
        # Generate year column
        df['year'] = year
        
        # Store the original DataFrame
        raw_provider_info[f'raw_pi_{year}'] = df

# Print out the keys of the dictionary to verify
print("Raw dataframes:", raw_provider_info.keys())

Successfully read the file with utf-8 encoding
Failed to read with utf-8 encoding
Successfully read the file with latin-1 encoding
Successfully read the file with utf-8 encoding
Failed to read with utf-8 encoding
Successfully read the file with latin-1 encoding
Failed to read with utf-8 encoding
Successfully read the file with latin-1 encoding
Successfully read the file with utf-8 encoding
Failed to read with utf-8 encoding
Successfully read the file with latin-1 encoding
Raw dataframes: dict_keys(['raw_pi_2016', 'raw_pi_2017', 'raw_pi_2015', 'raw_pi_2020', 'raw_pi_2021', 'raw_pi_2019', 'raw_pi_2018'])


In [200]:
# lowercase for all column names
for key in raw_provider_info:
    raw_provider_info[key].columns = raw_provider_info[key].columns.str.lower()

In [202]:
# Dictionary to store custom tables
clean_provider_info = {}

# Define the columns for each custom table
def create_provider_info_tables(raw_provider_info):
    tables = {}
    
    for key, df in raw_provider_info.items():
        year = key.split('_')[-1]
        
        # List the columns to keep
        # Adjust these column names to match actual data
        columns_to_keep = [
            'provnum', 'federal provider number',
            'provname', 'provider name',
            'address', 'provider address',
            'city', 'provider city',
            'state', 'provider state',
            'zip', 'provider zip code',
            'phone', 'provider phone number',
            'county_ssa', 'provider ssa county code',
            'county_name', 'provider county name',
            'ownership', 'ownership type',
            'bedcert', 'number of certified beds',
            'restot', 'average number of residents per day',
            'overall_rating', 'overall rating',
            'tot_penlty_cnt', 'total number of penalties',
            'rnhrd', 'reported rn staffing hours per resident per day',
            'totlichrd', 'reported licensed staffing hours per resident per day',
            'tothrd', 'reported total nurse staffing hours per resident per day',
            'pthrd', 'reported physical therapist staffing hours per resident per day',
            'year'
        ]
        
        # Only keep columns that exist in the dataframe
        valid_columns = [col for col in columns_to_keep if col in df.columns]
        
        # Create new table with only the columns you need
        if valid_columns:
            tables[f'provider_basic_{year}'] = df[valid_columns].copy()
    
    return tables

provider_info_tables = create_provider_info_tables(raw_provider_info)
clean_provider_info.update(provider_info_tables)

# Print out the keys of the dictionary to verify
print("Custom Tables:", clean_provider_info.keys())

Custom Tables: dict_keys(['provider_basic_2016', 'provider_basic_2017', 'provider_basic_2015', 'provider_basic_2020', 'provider_basic_2021', 'provider_basic_2019', 'provider_basic_2018'])


In [204]:
# Rename 2020 and 2021 file to standard name
for key, df in clean_provider_info.items():
    if key in ['provider_basic_2020', 'provider_basic_2021']:
        clean_provider_info[key] = df.rename(columns={
            'federal provider number': 'provnum',
            'provider name': 'provname',
            'provider address': 'address',
            'provider city': 'city',
            'provider state': 'state',
            'provider zip code': 'zip',
            'provider phone number': 'phone',
            'provider ssa county code': 'county_ssa',
            'provider county name': 'county_name',
            'ownership type': 'ownership',
            'number of certified beds': 'bedcert',
            'average number of residents per day': 'restot',
            'overall rating': 'overall_rating',
            'total number of penalties': 'tot_penlty_cnt',
            'reported rn staffing hours per resident per day': 'rnhrd',
            'reported licensed staffing hours per resident per day': 'totlichrd',
            'reported total nurse staffing hours per resident per day': 'tothrd',
            'reported physical therapist staffing hours per resident per day': 'pthrd'
        })

In [206]:
# Union all file
union_provider_info = pd.concat(clean_provider_info.values(), ignore_index=True)

union_provider_info.info()
union_provider_info.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108637 entries, 0 to 108636
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   provnum         108637 non-null  object 
 1   provname        108637 non-null  object 
 2   address         108637 non-null  object 
 3   city            108637 non-null  object 
 4   state           108637 non-null  object 
 5   zip             108637 non-null  int64  
 6   phone           108637 non-null  int64  
 7   county_ssa      108637 non-null  int64  
 8   county_name     108637 non-null  object 
 9   ownership       108636 non-null  object 
 10  bedcert         108637 non-null  int64  
 11  restot          107914 non-null  float64
 12  overall_rating  106945 non-null  float64
 13  tot_penlty_cnt  108637 non-null  int64  
 14  rnhrd           104754 non-null  float64
 15  totlichrd       104754 non-null  float64
 16  tothrd          104754 non-null  float64
 17  pthrd     

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year
39804,315271,CARNEYS POINT REHABILITATION AND NURSING CENTER,201 FIFTH AVENUE,CARNEYS POINT,NJ,8069,8562996800,340,Salem,For profit - Partnership,161,146.0,4.0,0,0.62568,1.46336,3.31506,0.06130,2015
33669,105930,VILLA HEALTH & REHABILITATION CENTER,120 CHIPOLA AVE,DELAND,FL,32720,3867383433,630,Volusia,For profit - Corporation,120,114.0,1.0,6,0.65395,1.65570,4.00395,0.10044,2015
92014,555839,DREIER'S NURSING CARE CENTER,1400 WEST GLENOAKS BLVD,GLENDALE,CA,91201,8182421183,200,Los Angeles,For profit - Corporation,53,47.4,2.0,1,1.44336,2.03596,4.81527,0.15010,2019


In [208]:
# Change data type

# Columns to convert to float
cols_to_convert = ['provnum', 'year']

# Convert specified columns to float
union_provider_info[cols_to_convert] = union_provider_info[cols_to_convert].apply(pd.to_numeric, errors='coerce').astype(float)

union_provider_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108637 entries, 0 to 108636
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   provnum         106923 non-null  float64
 1   provname        108637 non-null  object 
 2   address         108637 non-null  object 
 3   city            108637 non-null  object 
 4   state           108637 non-null  object 
 5   zip             108637 non-null  int64  
 6   phone           108637 non-null  int64  
 7   county_ssa      108637 non-null  int64  
 8   county_name     108637 non-null  object 
 9   ownership       108636 non-null  object 
 10  bedcert         108637 non-null  int64  
 11  restot          107914 non-null  float64
 12  overall_rating  106945 non-null  float64
 13  tot_penlty_cnt  108637 non-null  int64  
 14  rnhrd           104754 non-null  float64
 15  totlichrd       104754 non-null  float64
 16  tothrd          104754 non-null  float64
 17  pthrd     

In [210]:
# CLEAN COST REPORT

In [212]:
# Redefine the directory path
directory = '/Users/apple/Documents/APD/cost_report'

# Get all CSV files matching the pattern
files = glob.glob(os.path.join(directory, '*_CostReport.csv'))

# Dictionaries to store original DataFrames
raw_cost_report = {}

# Process each file
for file_path in files:
    # Extract the year from the filename
    filename = os.path.basename(file_path)
    year = filename.split('_')[0]
    
    # Read the CSV file
    df = read_csv_with_encodings(file_path)

    
    if df is not None:
        # Generate year column
        df['year'] = year
        
        # Store the original DataFrame
        raw_cost_report[f'raw_cost_{year}'] = df

# Print out the keys of the dictionary to verify
print("Raw dataframes:", raw_cost_report.keys())

Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Raw dataframes: dict_keys(['raw_cost_2021', 'raw_cost_2020', 'raw_cost_2017', 'raw_cost_2018', 'raw_cost_2019', 'raw_cost_2016', 'raw_cost_2015'])


In [214]:
# Lowercase for all column names
for key in raw_cost_report:
    raw_cost_report[key].columns = raw_cost_report[key].columns.str.lower()

In [216]:
# Dictionary to store your custom tables
clean_cost_report = {}

# Define the columns for each custom table
def create_cost_report_tables(raw_cost_report):
    tables = {}
    
    for key, df in raw_cost_report.items():
        year = key.split('_')[-1]
        
        # List the columns to keep
        # Adjust these column names to match actual data
        columns_to_keep = [
            'provider_ccn', 'provider ccn',
            'rural_versus_urban', 'rural versus urban',
            'gross_revenue', 'gross revenue',
            'inpatient_revenue', 'inpatient revenue',
            'net_income', 'net income',
            'net_patient_revenue', 'net patient revenue',
            'number_of_beds', 'number of beds',
            'total_costs', 'total costs',
            'total_income', 'total income',
            'total_salaries_adjusted', 'total salaries (adjusted)',
            'fiscal_year_begin_date', 'fiscal_year_end_date',
            'year'
        ]

        
        # Only keep columns that exist in the dataframe
        valid_columns = [col for col in columns_to_keep if col in df.columns]
        
        # Create new table
        if valid_columns:
            tables[f'cost_report_clean_{year}'] = df[valid_columns].copy()
    
    return tables

cost_report_tables = create_cost_report_tables(raw_cost_report)
clean_cost_report.update(cost_report_tables)

# Print out the keys of the dictionary to verify
print("Custom Tables:", clean_cost_report.keys())

Custom Tables: dict_keys(['cost_report_clean_2021', 'cost_report_clean_2020', 'cost_report_clean_2017', 'cost_report_clean_2018', 'cost_report_clean_2019', 'cost_report_clean_2016', 'cost_report_clean_2015'])


In [218]:
# Rename 2020 and 2021 file to standard name
for key, df in clean_cost_report.items():
    if key in ['cost_report_clean_2020', 'cost_report_clean_2021']:
        clean_cost_report[key] = df.rename(columns={
            'provider ccn': 'provider_ccn',
            'rural versus urban': 'rural_versus_urban',
            'gross revenue': 'gross_revenue',
            'inpatient revenue': 'inpatient_revenue',
            'net income': 'net_income',
            'net patient revenue': 'net_patient_revenue',
            'number of beds': 'number_of_beds',
            'total costs': 'total_costs',
            'total income': 'total_income',
            'total salaries (adjusted)': 'total_salaries_adjusted'
        })

In [220]:
# Union all file
union_cost_report = pd.concat(clean_cost_report.values(), ignore_index=True)

union_cost_report.info()
union_cost_report.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106269 entries, 0 to 106268
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   provider_ccn             106269 non-null  int64  
 1   rural_versus_urban       104082 non-null  object 
 2   gross_revenue            103988 non-null  float64
 3   inpatient_revenue        103985 non-null  float64
 4   net_income               89205 non-null   float64
 5   net_patient_revenue      103997 non-null  float64
 6   number_of_beds           103867 non-null  float64
 7   total_costs              104131 non-null  float64
 8   total_income             103981 non-null  float64
 9   total_salaries_adjusted  104196 non-null  float64
 10  year                     106269 non-null  object 
 11  fiscal_year_begin_date   74602 non-null   object 
 12  fiscal_year_end_date     74602 non-null   object 
dtypes: float64(8), int64(1), object(4)
memory usage: 10.5+ MB


,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,year,fiscal_year_begin_date,fiscal_year_end_date
43299,255305,R,7730633.0,7730633.0,942877.0,6321528.0,60.0,968239.0,942877.0,2173131.0,2017,1/1/2017,12/31/2017
46327,75163,U,21004515.0,21004515.0,-179471.0,14780538.0,120.0,1695474.0,-179471.0,7117914.0,2018,10/1/2017,9/30/2018
52166,245257,R,9693677.0,9693677.0,-59215.0,8620549.0,93.0,671702.0,-59230.0,4516648.0,2018,1/1/2018,12/31/2018


In [222]:
# Change data type

# Columns to convert to float
cols_to_convert = ['provider_ccn', 'year']

# Convert specified columns to float
union_cost_report[cols_to_convert] = union_cost_report[cols_to_convert].apply(pd.to_numeric, errors='coerce').astype(float)
union_cost_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106269 entries, 0 to 106268
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   provider_ccn             106269 non-null  float64
 1   rural_versus_urban       104082 non-null  object 
 2   gross_revenue            103988 non-null  float64
 3   inpatient_revenue        103985 non-null  float64
 4   net_income               89205 non-null   float64
 5   net_patient_revenue      103997 non-null  float64
 6   number_of_beds           103867 non-null  float64
 7   total_costs              104131 non-null  float64
 8   total_income             103981 non-null  float64
 9   total_salaries_adjusted  104196 non-null  float64
 10  year                     106269 non-null  float64
 11  fiscal_year_begin_date   74602 non-null   object 
 12  fiscal_year_end_date     74602 non-null   object 
dtypes: float64(10), object(3)
memory usage: 10.5+ MB


In [224]:
# LOAD AND CLEAN HD

In [226]:
# Redefine the directory path
directory = '/Users/apple/Documents/APD/HD'

# Get all CSV files matching the pattern
files = glob.glob(os.path.join(directory, 'HD*.csv'))

# Dictionaries to store original DataFrames
raw_HD = {}

# Process each file
for file_path in files:
    # Extract the year from the filename
    filename = os.path.basename(file_path)
    year = filename[2:6]
    
    # Read the CSV file
    df = read_csv_with_encodings(file_path)

    
    if df is not None:
        # Generate year column
        df['year'] = year
        
        # Store the original DataFrame
        raw_HD[f'raw_HD{year}'] = df

# Print out the keys of the dictionary to verify
print("Raw dataframes:", raw_HD.keys())

Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Successfully read the file with utf-8 encoding
Raw dataframes: dict_keys(['raw_HD2019', 'raw_HD2018', 'raw_HD2020', 'raw_HD2021', 'raw_HD2016', 'raw_HD2017', 'raw_HD2015'])


In [228]:
# Standardize Column Name
for key in raw_HD:
    raw_HD[key].columns = raw_HD[key].columns.str.lower()

for key in raw_HD:
    raw_HD[key] = raw_HD[key].rename(columns = {
        'sum of hd code': 'sum_of_hd_code',
        'count of hd code': 'count_of_hd_code'
    })

In [230]:
# Union all file
union_HD = pd.concat(raw_HD.values(), ignore_index=True)

union_HD.info()
union_HD.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107516 entries, 0 to 107515
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   provnum           107516 non-null  object
 1   sum_of_hd_code    107516 non-null  int64 
 2   count_of_hd_code  107516 non-null  int64 
 3   year              107516 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.3+ MB


,provnum,sum_of_hd_code,count_of_hd_code,year
104837,495374,133,33,2015
62646,75352,66,15,2016
94656,115671,49,11,2015


In [232]:
# Change data type

# Columns to convert to float
cols_to_convert = ['provnum', 'year']

# Convert specified columns to float
union_HD[cols_to_convert] = union_HD[cols_to_convert].apply(pd.to_numeric, errors='coerce').astype(float)

union_HD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107516 entries, 0 to 107515
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   provnum           105836 non-null  float64
 1   sum_of_hd_code    107516 non-null  int64  
 2   count_of_hd_code  107516 non-null  int64  
 3   year              107516 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 3.3 MB


In [234]:
# MERGE ALL FILE 

In [236]:
nursing_merge = union_provider_info.merge(union_cost_report, 
                                          left_on=['provnum', 'year'], 
                                          right_on=['provider_ccn', 'year'], 
                                          how='right')
nursing_merge = nursing_merge.merge(union_HD, 
                      left_on=['provider_ccn', 'year'], 
                      right_on=['provnum', 'year'], 
                      how='left')
nursing_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106269 entries, 0 to 106268
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   provnum_x                105832 non-null  float64
 1   provname                 105832 non-null  object 
 2   address                  105832 non-null  object 
 3   city                     105832 non-null  object 
 4   state                    105832 non-null  object 
 5   zip                      105832 non-null  float64
 6   phone                    105832 non-null  float64
 7   county_ssa               105832 non-null  float64
 8   county_name              105832 non-null  object 
 9   ownership                105831 non-null  object 
 10  bedcert                  105832 non-null  float64
 11  restot                   105260 non-null  float64
 12  overall_rating           104168 non-null  float64
 13  tot_penlty_cnt           105832 non-null  float64
 14  rnhr

In [238]:
# Remove duplicate rows and columns
nursing_merge = nursing_merge.drop_duplicates()
nursing_merge = nursing_merge.drop(columns = 'provnum_y')
nursing_merge = nursing_merge.rename(columns={'provnum_x':'provnum'})


nursing_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106228 entries, 0 to 106268
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   provnum                  105793 non-null  float64
 1   provname                 105793 non-null  object 
 2   address                  105793 non-null  object 
 3   city                     105793 non-null  object 
 4   state                    105793 non-null  object 
 5   zip                      105793 non-null  float64
 6   phone                    105793 non-null  float64
 7   county_ssa               105793 non-null  float64
 8   county_name              105793 non-null  object 
 9   ownership                105792 non-null  object 
 10  bedcert                  105793 non-null  float64
 11  restot                   105224 non-null  float64
 12  overall_rating           104135 non-null  float64
 13  tot_penlty_cnt           105793 non-null  float64
 14  rnhrd    

In [240]:
# Clean data type in merging file

# Convert specified columns to string
#cols_to_convert = ['provnum', 'year','provider_ccn']
#nursing_merge[cols_to_convert] = nursing_merge[cols_to_convert].apply(pd.to_numeric, errors='coerce').astype(int)

cols_to_convert = ['fiscal_year_begin_date', 'fiscal_year_end_date']
nursing_merge[cols_to_convert] = nursing_merge[cols_to_convert].apply(pd.to_datetime, errors='coerce')
nursing_merge['phone'] = nursing_merge['phone'].astype(str)

nursing_merge.info()
nursing_merge.sample(3)

<class 'pandas.core.frame.DataFrame'>
Index: 106228 entries, 0 to 106268
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   provnum                  105793 non-null  float64       
 1   provname                 105793 non-null  object        
 2   address                  105793 non-null  object        
 3   city                     105793 non-null  object        
 4   state                    105793 non-null  object        
 5   zip                      105793 non-null  float64       
 6   phone                    106228 non-null  object        
 7   county_ssa               105793 non-null  float64       
 8   county_name              105793 non-null  object        
 9   ownership                105792 non-null  object        
 10  bedcert                  105793 non-null  float64       
 11  restot                   105224 non-null  float64       
 12  overall_rating       

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,fiscal_year_begin_date,fiscal_year_end_date,sum_of_hd_code,count_of_hd_code
45185,395564.0,RIVERSIDE REHABILITATION AND NURSING CENTER,500 WEST HOSPITAL STREET,TAYLOR,PA,18517.0,5705622102.0,420.0,Lackawanna,For profit - Corporation,161.0,142.0,1.0,2.0,0.58451,1.42746,3.47465,1.23239,2017.0,395564.0,U,15836942.0,15836942.0,47355.0,5807813.0,161.0,783893.0,47355.0,2057250.0,2017-08-01,2017-12-31,206.0,44.0
105511,675447.0,THE HIGHLANDS OF DALLAS,9009 FOREST LN,DALLAS,TX,75243.0,9727831771.0,390.0,Dallas,For profit - Individual,116.0,92.0,3.0,0.0,0.37120,1.31467,3.36848,0.03424,2015.0,675447.0,U,NaN,NaN,-4752899.0,NaN,120.0,56770.0,-4752899.0,2632831.0,2015-01-01,2015-12-31,138.0,29.0
30767,675013.0,CROWELL NURSING CENTER,200 SOUTH B AVE,CROWELL,TX,79227.0,9406841511.0,522.0,Foard,Government - Hospital district,67.0,35.0,5.0,0.0,0.46286,0.46286,2.84715,0.03429,2017.0,675013.0,R,716881.0,716881.0,-41651.0,532377.0,80.0,111375.0,-41651.0,214126.0,2017-01-01,2017-03-31,26.0,5.0


In [242]:
# Check category
rvu = nursing_merge['rural_versus_urban'].unique()
print(f'rvu: {rvu}')

state_cat = nursing_merge['state'].unique()
print(f'state: {state_cat}')

ownership_cat = nursing_merge['ownership'].unique()
print(f'ownership: {ownership_cat}')

rvu: ['U' 'R' nan]
state: ['NH' nan 'MO' 'MT' 'OK' 'TX' 'WA' 'LA' 'IL' 'WI' 'FL' 'OH' 'PA' 'KY' 'MI'
 'CO' 'CA' 'KS' 'NC' 'CT' 'IA' 'NJ' 'RI' 'IN' 'TN' 'VA' 'MA' 'MN' 'NE'
 'MD' 'OR' 'ID' 'GA' 'AL' 'WV' 'VT' 'SC' 'NM' 'AR' 'UT' 'ME' 'AZ' 'ND'
 'MS' 'SD' 'AK' 'NV' 'NY' 'HI' 'WY' 'DE' 'PR' 'DC']
ownership: ['For profit - Corporation' nan 'Government - City'
 'For profit - Limited Liability company' 'For profit - Partnership'
 'Government - Hospital district' 'For profit - Individual'
 'Government - State' 'Non profit - Corporation' 'Non profit - Other'
 'Non profit - Church related' 'Government - County'
 'Government - Federal' 'Government - City/county']


In [244]:
# Test a random sample with missing values in provider info 
test = nursing_merge[nursing_merge['provider_ccn']== 675056.0]
pd.set_option('display.max_columns', None)
display(test)

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,fiscal_year_begin_date,fiscal_year_end_date,sum_of_hd_code,count_of_hd_code
32916,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,40.0,4.0,3.0,2.05000,2.40750,4.48625,0.14250,2017.0,675056.0,U,2776674.0,2776674.0,-19994.0,2563479.0,65.0,351624.0,-19994.0,1276714.0,2017-01-01,2017-12-31,134.0,26.0
46503,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Partnership,57.0,32.9,3.0,0.0,0.53941,1.19013,3.19712,0.05137,2018.0,675056.0,U,2250912.0,2250912.0,-424616.0,2111356.0,65.0,451556.0,-424614.0,1169914.0,2018-01-01,2018-12-31,103.0,20.0
60751,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,675056.0,U,998856.0,998856.0,-289611.0,873973.0,57.0,219103.0,-289611.0,554644.0,2019-01-01,2019-07-05,NaN,NaN
78039,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,41.0,1.0,3.0,0.69390,0.83293,2.81097,0.13902,2016.0,675056.0,R,2969912.0,2969912.0,-16998.0,2706109.0,65.0,411467.0,-17000.0,1345560.0,2016-01-01,2016-12-31,148.0,30.0
92517,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,61.0,42.0,1.0,2.0,0.33571,0.97500,2.64167,0.00000,2015.0,675056.0,U,2416318.0,2416318.0,-302684.0,2214843.0,65.0,339184.0,-302684.0,1165727.0,2015-01-01,2015-12-31,219.0,47.0


In [246]:
import numpy as np

def fill_missing_provider_info(df):
    """
    Fill in missing provider infomation by looking up the same provider_ccn from previous years.
    Args: df (pandas.DataFrame): DataFrame containing provider information with some rows missing data
    Returns: pandas.DataFrame: DataFrame with missing provider information filled in
    """
    # Make a copy of the input df
    filled_nursing = nursing_merge.copy()

    # Convert provider_ccn to string to ensure consistent type
    filled_nursing['provider_ccn'] = filled_nursing['provider_ccn'].astype(str)

    # Create reference dict of provider details
    provider_details = {}

    # Columns to fill
    provider_cols = ['provnum','provname','address','city','state','zip','phone','county_ssa','county_name','ownership']

    # Collect all available provider details
    for idx, row in filled_nursing.iterrows():
        provider_ccn = row['provider_ccn']
        if pd.isna(provider_ccn) or provider_ccn == '':
            continue
        # Create dict with values for provider columns
        details = {col: row[col] for col in provider_cols if col in filled_nursing.columns and not pd.isna(row[col])}
        # Add detilas to provider details dict if it doesn't exist or has fewer non-null values
        if provider_ccn not in provider_details or len(details) > len(provider_details[provider_ccn]):
            provider_details[provider_ccn] = details

    # Fill in missing values
    for idx, row in filled_nursing.iterrows():
        provider_ccn = row['provider_ccn']
        if pd.isna(provider_ccn) or provider_ccn == '':
            continue
        if provider_ccn in provider_details:
            for col in provider_cols:
                if col in filled_nursing.columns and (
                    pd.isna(filled_nursing.at[idx, col])
                    or filled_nursing.at[idx, col] == '' or
                    filled_nursing.at[idx, col] == '' or
                    filled_nursing.at[idx, col] == 'nan' or
                    str(filled_nursing.at[idx, col]).strip() == '' or
                    str(filled_nursing.at[idx, col]).lower() == 'nan'
                ):
                    if col in provider_details[provider_ccn]:
                        filled_nursing.at[idx, col] = provider_details[provider_ccn][col]
    return filled_nursing

In [248]:
filled_nursing = fill_missing_provider_info(nursing_merge)

In [250]:
filled_nursing.info()
filled_nursing.sample(3)

<class 'pandas.core.frame.DataFrame'>
Index: 106228 entries, 0 to 106268
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   provnum                  106160 non-null  float64       
 1   provname                 106160 non-null  object        
 2   address                  106160 non-null  object        
 3   city                     106160 non-null  object        
 4   state                    106160 non-null  object        
 5   zip                      106160 non-null  float64       
 6   phone                    106228 non-null  object        
 7   county_ssa               106160 non-null  float64       
 8   county_name              106160 non-null  object        
 9   ownership                106160 non-null  object        
 10  bedcert                  105793 non-null  float64       
 11  restot                   105224 non-null  float64       
 12  overall_rating       

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,fiscal_year_begin_date,fiscal_year_end_date,sum_of_hd_code,count_of_hd_code
60168,345492.0,NC STATE VETERANS HOME - FAYETTEVILLE,214 COCHRAN AVENUE,FAYETTEVILLE,NC,28301.0,9104824131.0,250.0,Cumberland,Government - State,150.0,141.8,5.0,1.0,0.49381,1.47777,3.99282,0.05468,2018.0,345492.0,U,14670266.0,14670266.0,-13700014.0,12358988.0,150.0,3347587.0,-13700014.0,7238253.0,2018-07-01,2019-06-30,40.0,10.0
18522,525628.0,MARINUKA MANOR,19475 SILVER CREEK RD,GALESVILLE,WI,54630.0,6085822211.0,600.0,Trempealeau,Government - City,59.0,31.8,5.0,0.0,1.02269,1.73835,3.96342,0.05524,2020.0,525628.0,R,2984426.0,3194301.0,NaN,-525018.0,59.0,220276.0,554469.0,2006429.0,NaT,NaT,45.0,11.0
11843,335632.0,PINE HAVEN HOME,NY ROUTE 217,PHILMONT,NY,12565.0,5186724021.0,200.0,Columbia,For profit - Corporation,120.0,87.9,2.0,0.0,0.63747,1.65926,3.38463,0.11925,2021.0,335632.0,U,14859086.0,14859086.0,1508699.0,11832231.0,120.0,1496253.0,1526654.0,5466421.0,NaT,NaT,95.0,22.0


In [252]:
# Test with the previous sample
test = filled_nursing[filled_nursing['provider_ccn']=="675056.0"]
pd.set_option('display.max_columns', None)
display(test)

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,fiscal_year_begin_date,fiscal_year_end_date,sum_of_hd_code,count_of_hd_code
32916,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,40.0,4.0,3.0,2.05000,2.40750,4.48625,0.14250,2017.0,675056.0,U,2776674.0,2776674.0,-19994.0,2563479.0,65.0,351624.0,-19994.0,1276714.0,2017-01-01,2017-12-31,134.0,26.0
46503,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Partnership,57.0,32.9,3.0,0.0,0.53941,1.19013,3.19712,0.05137,2018.0,675056.0,U,2250912.0,2250912.0,-424616.0,2111356.0,65.0,451556.0,-424614.0,1169914.0,2018-01-01,2018-12-31,103.0,20.0
60751,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,675056.0,U,998856.0,998856.0,-289611.0,873973.0,57.0,219103.0,-289611.0,554644.0,2019-01-01,2019-07-05,NaN,NaN
78039,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,41.0,1.0,3.0,0.69390,0.83293,2.81097,0.13902,2016.0,675056.0,R,2969912.0,2969912.0,-16998.0,2706109.0,65.0,411467.0,-17000.0,1345560.0,2016-01-01,2016-12-31,148.0,30.0
92517,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,61.0,42.0,1.0,2.0,0.33571,0.97500,2.64167,0.00000,2015.0,675056.0,U,2416318.0,2416318.0,-302684.0,2214843.0,65.0,339184.0,-302684.0,1165727.0,2015-01-01,2015-12-31,219.0,47.0


In [254]:
# Same sample, the fiscal year in each report is different -> take all value/day in fiscal year * 365

# Column to adjust
adjust_cols = ['gross_revenue','inpatient_revenue','net_income','net_patient_revenue','total_costs','total_salaries_adjusted','total_income']

# Adjust value to column
'''
for idx, row in filled_nursing.iterrows():
    filled_nursing['fiscal_period'] = (filled_nursing['fiscal_year_end_date'] - filled_nursing['fiscal_year_begin_date']).dt.days
    for col in adjust_cols:
        filled_nursing[col + '_annualized'] = (filled_nursing[col] / filled_nursing['fiscal_period']) * 365
'''
filled_nursing['fiscal_period_days'] = (filled_nursing['fiscal_year_end_date'] - filled_nursing['fiscal_year_begin_date']).dt.days + 1

for col in adjust_cols:
    filled_nursing[col + '_annualized'] = filled_nursing[col] * 365 / filled_nursing['fiscal_period_days']

In [256]:
# Test with the previous sample
test = filled_nursing[filled_nursing['provider_ccn']=="675056.0"]
pd.set_option('display.max_columns', None)
display(test)

,provnum,provname,address,city,state,zip,phone,county_ssa,county_name,ownership,bedcert,restot,overall_rating,tot_penlty_cnt,rnhrd,totlichrd,tothrd,pthrd,year,provider_ccn,rural_versus_urban,gross_revenue,inpatient_revenue,net_income,net_patient_revenue,number_of_beds,total_costs,total_income,total_salaries_adjusted,fiscal_year_begin_date,fiscal_year_end_date,sum_of_hd_code,count_of_hd_code,fiscal_period_days,gross_revenue_annualized,inpatient_revenue_annualized,net_income_annualized,net_patient_revenue_annualized,total_costs_annualized,total_salaries_adjusted_annualized,total_income_annualized
32916,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,40.0,4.0,3.0,2.05000,2.40750,4.48625,0.14250,2017.0,675056.0,U,2776674.0,2776674.0,-19994.0,2563479.0,65.0,351624.0,-19994.0,1276714.0,2017-01-01,2017-12-31,134.0,26.0,365.0,2.776674e+06,2.776674e+06,-19994.000000,2.563479e+06,351624.000000,1.276714e+06,-19994.000000
46503,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Partnership,57.0,32.9,3.0,0.0,0.53941,1.19013,3.19712,0.05137,2018.0,675056.0,U,2250912.0,2250912.0,-424616.0,2111356.0,65.0,451556.0,-424614.0,1169914.0,2018-01-01,2018-12-31,103.0,20.0,365.0,2.250912e+06,2.250912e+06,-424616.000000,2.111356e+06,451556.000000,1.169914e+06,-424614.000000
60751,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,675056.0,U,998856.0,998856.0,-289611.0,873973.0,57.0,219103.0,-289611.0,554644.0,2019-01-01,2019-07-05,NaN,NaN,186.0,1.960121e+06,1.960121e+06,-568322.661290,1.715055e+06,429960.188172,1.088414e+06,-568322.661290
78039,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,57.0,41.0,1.0,3.0,0.69390,0.83293,2.81097,0.13902,2016.0,675056.0,R,2969912.0,2969912.0,-16998.0,2706109.0,65.0,411467.0,-17000.0,1345560.0,2016-01-01,2016-12-31,148.0,30.0,366.0,2.961797e+06,2.961797e+06,-16951.557377,2.698715e+06,410342.773224,1.341884e+06,-16953.551913
92517,675056.0,STANTON NURSING AND REHABILITATION LP,1100 W BROADWAY,STANTON,TX,79782.0,4327562841.0,784.0,Martin,For profit - Corporation,61.0,42.0,1.0,2.0,0.33571,0.97500,2.64167,0.00000,2015.0,675056.0,U,2416318.0,2416318.0,-302684.0,2214843.0,65.0,339184.0,-302684.0,1165727.0,2015-01-01,2015-12-31,219.0,47.0,365.0,2.416318e+06,2.416318e+06,-302684.000000,2.214843e+06,339184.000000,1.165727e+06,-302684.000000


In [258]:
# Export to csv
file_path = os.path.join(os.getcwd(), '/Users/apple/Documents/APD/','nursing_clean.csv')
filled_nursing.to_csv(file_path, index=False)